# Getting Started

## Introduction
ReSyPE (pronounced *recipe*) is a Python library built for both practitioners and researchers that wish to quickly develop and deploy ML-based recommender systems.

The library provides an end-to-end pipeline that includes:

1. Loading transaction, user feature, and item feature datasets
2. Interchangable methods for user and item clustering
3. Modular framework for machine learning models
4. Iterative and decomposition-based techniques


## Installation

`pip install resype`


## Load the Data

In [1]:
import pandas as pd
import numpy as np
from resype.resype import Resype

%load_ext autoreload
%autoreload 2 

In [2]:
# load transaction list
transaction_list = pd.read_csv("sample_data/ratings.csv")[['userId', 'movieId', 'rating']]
transaction_list = transaction_list.sample(20)
transaction_list.columns = ["user_id", 'item_id', 'rating']

## Preprocess

In [3]:
re = Resype(transaction_list)
re.transaction_list.head(3)

,user_id,item_id,rating
90623,590,922,4.0
67969,438,5349,4.0
95263,600,1042,2.5


In [4]:
# construct utlity matrix
re.construct_utility_matrix()
re.utility_matrix.head(3)

item_id,104,500,586,849,904,922,1042,1213,2762,3646,4574,4993,5349,5952,52767,52973,54259,58870,91630,136864
user_id,,,,,,,,,,,,,,,,,,,,
16,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Train on Unclustered Matrix

### Iterative Approach

In [5]:
# import sklearn Model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [6]:
# fit and predict
re.fit(model, method='iterative', return_models=False)
re.utility_matrix_preds.head(3)

item_id,104,500,586,849,904,922,1042,1213,2762,3646,4574,4993,5349,5952,52767,52973,54259,58870,91630,136864
user_id,,,,,,,,,,,,,,,,,,,,
16,0.0,0.0,0.0,0.25,-0.25,0.0,0.0,0.0,0.0,-0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0
17,0.0,0.0,0.0,0.25,-0.25,0.0,0.0,0.0,0.0,-0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0
20,0.0,0.0,0.0,0.25,-0.25,0.0,0.0,0.0,0.0,-0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0


In [7]:
# recommend
user_list = [0, 1, 2] # indices
top_n = 10
re.get_rec(user_list, top_n)
re.df_rec

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,16,58870.0,136864.0,91630.0,500.0,586.0,922.0,1042.0,1213.0,2762.0,104.0
1,17,58870.0,849.0,136864.0,91630.0,500.0,586.0,922.0,1042.0,1213.0,104.0
2,20,58870.0,849.0,136864.0,91630.0,500.0,586.0,922.0,1042.0,1213.0,2762.0


## Train on Clustered matrix

In [8]:
from sklearn.cluster import KMeans
km_users = KMeans(n_clusters=10)
km_items = KMeans(n_clusters=10)

user_model, user_cluster_map, util_matrix_w_users = re.cluster_users(km_users)
item_model, item_cluster_map, util_matrix_w_items = re.cluster_items(km_items)

In [9]:
re.cluster_assignment()
re.utility_matrix_agg(u_agg="mean", i_agg="mean")
re.utility_matrix.head(3)

i_cluster,0,1,2,3,4,5,6,7,8,9
u_cluster,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.ensemble import RandomForestRegressor
model1 = RandomForestRegressor()
re.fit(model_object=model1, method='iterative', n_synth_data=5, p=0.1)
re.utility_matrix_preds.head(3)

i_cluster,0,1,2,3,4,5,6,7,8,9
u_cluster,,,,,,,,,,
0,-0.408889,-0.408889,-0.408889,-0.408889,-0.408889,3.591111,-0.408889,-0.295889,-0.408889,-0.408889
1,-0.028667,0.181883,-0.091494,-0.028667,-0.096944,-0.028667,-0.028667,-0.130317,0.118933,-0.028667
2,-0.470000,-0.470000,4.030000,-0.447367,-0.451738,-0.470000,-0.470000,-0.470000,-0.470000,-0.470000


In [11]:
# predict top item clusters per user index
user_list = [0, 1, 2] # index
top_n = 5 # top n clusters
re.get_rec(user_list, top_n, re.user_assignment)

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5
0,0,9.0,0.0,8.0,7.0,6.0
1,1,8.0,9.0,6.0,5.0,3.0
2,2,0.0,9.0,6.0,8.0,7.0


In [12]:
# predict top items per user_id
top_n = 5 # top n clusters
re.get_rec_item(top_n)

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5
0,16,52973.0,1213.0,5349.0,52767.0,922.0
1,17,5349.0,52973.0,922.0,91630.0,904.0
2,20,1213.0,52973.0,922.0,5349.0,52767.0
